In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import sys
from darts import SeasonalityMode
from darts.models import NaiveSeasonal, NaiveDrift
from darts.models import Prophet, ExponentialSmoothing, AutoARIMA, Theta, FourTheta
from darts.models import LinearRegressionModel
from darts.utils.statistics import check_seasonality, remove_seasonality, extract_trend_and_seasonality

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm

import pickle

In [3]:
from M4_metrics import owa_m4, smape_m4, mase_m4

In [4]:
from darts.utils.timeseries_generation import constant_timeseries as ct

In [5]:
import logging
import warnings
warnings.filterwarnings('ignore')
logging.disable(logging.CRITICAL)

In [6]:
download = True
preprocess = True

### Download and create TimeSeries

In [ ]:
if download:
    %run -i "download_data_M4.py"
if preprocess:
    %run -i "create_ts.py"

M4-info already exists
Yearly-train already exists
Yearly-test already exists
Quarterly-train already exists
Quarterly-test already exists
Monthly-train already exists
Monthly-test already exists
Weekly-train already exists
Weekly-test already exists
Daily-train already exists
Daily-test already exists
Hourly-train already exists
Hourly-test already exists


### dataset info

In [ ]:
# data_categories = ['Macro', 'Micro', 'Demographic', 'Industry', 'Finance', 'Other']
data_freq = ['Yearly', 'Quarterly', 'Monthly', 'Weekly', 'Daily', 'Hourly']
info_dataset = pd.read_csv('dataset/M4-info.csv', delimiter=',').set_index('M4id')

In [ ]:
info_dataset.head()

In [ ]:
info_dataset.filter(regex='W', axis=0).category.unique()

In [ ]:
info_dataset.groupby('SP').count()

In [ ]:
print(info_dataset.groupby('SP').Frequency.unique())
print(info_dataset.groupby('SP').Horizon.unique())

### evaluating methods 

In [ ]:
freq = 'Hourly'
ts_train = pickle.load(open("dataset/train_"+freq+".pkl", "rb"))
ts_test = pickle.load(open("dataset/test_"+freq+".pkl", "rb"))

In [ ]:
mase_all = []
smape_all = []
m = int(info_dataset.Frequency[freq[0]+'1'])
for train, test in tqdm(zip(ts_train, ts_test)):
    # remove seasonality
    train_des=train
    seasonOut = ct(length=len(test), freq=train.freq_str, start_ts=test.start_time())
    season = ct(length=len(train), freq=train.freq_str, start_ts=train.start_time())
    if m > 1:
        if check_seasonality(train, m=m, max_lag=2*m):
            _, season = extract_trend_and_seasonality(train, m, model=SeasonalityMode.MULTIPLICATIVE)
            train_des = remove_seasonality(train, freq=m, model=SeasonalityMode.MULTIPLICATIVE)
            seasonOut = season[-m:].shift(m)
            seasonOut = seasonOut.append_values(seasonOut.values())[:len(test)]
    # model selection
    naiveSeason = NaiveSeasonal(K=m)
    naive2 = NaiveSeasonal(K=1)
    ses = ExponentialSmoothing(trend=None, seasonal=None)
    holt = ExponentialSmoothing(seasonal=None, damped=False, trend='additive')
    damp = ExponentialSmoothing(seasonal=None, damped=True, trend='additive')
    fourtheta = FourTheta.select_best_model(train, thetas=[1,2,3], m=m)
    # model fitting
    naiveSeason.fit(train)
    naive2.fit(train_des)
    fourtheta.fit(train)
    ses.fit(train_des)
    holt.fit(train_des)
    damp.fit(train_des)
    # forecasting
    forecast_naiveSeason = naiveSeason.predict(len(test))
    forecast_naive2 = naive2.predict(len(test)) * seasonOut
    forecast_fourtheta = fourtheta.predict(len(test))
    forecast_ses = ses.predict(len(test))*seasonOut
    forecast_holt = holt.predict(len(test))*seasonOut
    forecast_damp = damp.predict(len(test))*seasonOut
    # baseline constant weight ensembling
    forecast_comb = ((forecast_ses + forecast_holt + forecast_damp) / 3)
    
    mase_all.append(np.vstack([
                               mase_m4(train, test, forecast_naiveSeason, m=m),
                               mase_m4(train, test, forecast_naive2, m=m),
                               mase_m4(train, test, forecast_fourtheta, m=m),
                               mase_m4(train, test, forecast_ses, m=m),
                               mase_m4(train, test, forecast_holt, m=m),
                               mase_m4(train, test, forecast_damp, m=m),
                               mase_m4(train, test, forecast_comb, m=m),
                              ]))
    smape_all.append(np.vstack([
                                smape_m4(test, forecast_naiveSeason),
                                smape_m4(test, forecast_naive2),
                                smape_m4(test, forecast_fourtheta),
                                smape_m4(test, forecast_ses),
                                smape_m4(test, forecast_holt),
                                smape_m4(test, forecast_damp),
                                smape_m4(test, forecast_comb),
                               ]))

In [ ]:
print("MASE; naiveSeason: {:.3f}, naive2: {:.3f}, 4Theta: {:.3f},\n"
       "SES: {:.3f}, Holt: {:.3f}, Damp: {:.3f}, Comb: {:.3f}\n".format(*tuple(np.stack(mase_all).mean(axis=(0,2)))))
print("sMAPE; naiveSeason: {:.3f}, naive2: {:.3f}, 4Theta: {:.3f},\n"
       "SES: {:.3f}, Holt: {:.3f}, Damp: {:.3f}, Comb: {:.3f}\n".format(*tuple(np.stack(smape_all).mean(axis=(0,2)))))
print("OWA; naiveSeason: {:.3f}, naive2: {:.3f}, 4Theta: {:.3f},\n"
       "SES: {:.3f}, Holt: {:.3f}, Damp: {:.3f}, Comb: {:.3f}\n".format(*tuple(owa_m4(freq, 
                                                                        np.stack(smape_all).mean(axis=(0,2)), 
                                                                        np.stack(mase_all).mean(axis=(0,2))))))

In [ ]:
train[-2*m:].plot(label='train')
test.plot(label='test')
forecast_naiveSeason.plot(label='naive seasonal')
forecast_naive2.plot(label='naive2')
plt.legend()

In [ ]:
plt.hist(np.nanmean(np.stack(mase_all), axis=(2,))[:,3], bins=100, label='4Theta')
plt.hist(np.nanmean(np.stack(mase_all), axis=(2,))[:,0], bins=30, label='naiveSeason', alpha=0.7)
plt.legend();

Visualization

In [ ]:
freq = 'Monthly'
ts_train = pickle.load(open("dataset/train_"+freq+".pkl", "rb"))
ts_test = pickle.load(open("dataset/test_"+freq+".pkl", "rb"))

In [ ]:
_id = 1
train = ts_train[_id]
test = ts_test[_id]

In [ ]:
m = int(info_dataset.Frequency[freq[0]+'1'])
train_des=train
seasonOut = ct(length=len(test), freq=train.freq_str, start_ts=test.start_time())
season = ct(length=len(train), freq=train.freq_str, start_ts=train.start_time())
if m > 1:
    if check_seasonality(train, m=int(m), max_lag=2*m):
        _, season = extract_trend_and_seasonality(train, m, model=SeasonalityMode.MULTIPLICATIVE)
        train_des = remove_seasonality(train, freq=m, model=SeasonalityMode.MULTIPLICATIVE)
        seasonOut = season[-m:].shift(m)
        seasonOut = seasonOut.append_values(seasonOut.values())[:len(test)]
    else:
        m = 1
# model selection
naiveSeason = NaiveSeasonal(K=m)
naive2 = NaiveSeasonal(K=1)
prophet = Prophet(yearly_seasonality=True, changepoint_range=0.95)
arima = AutoARIMA()
fourtheta = FourTheta.select_best_model(train, thetas=[1, 2, 3], m=m)
# model fitting
naiveSeason.fit(train)
naive2.fit(train_des)
fourtheta.fit(train)
prophet.fit(train)
arima.fit(train)
# forecasting
forecast_naiveSeason = naiveSeason.predict(len(test))
forecast_naive2 = naive2.predict(len(test)) * seasonOut
forecast_fourtheta = fourtheta.predict(len(test))
forecast_arima = arima.predict(len(test))
forecast_prophet = prophet.predict(len(test))

In [ ]:
train[-m:].plot(label='train')
test.plot(label='test', lw=3)
forecast_naiveSeason.plot(label='naiveS')
forecast_fourtheta.plot(label='4theta')
forecast_naive2.plot(label='naive2')
forecast_arima.plot(label='ARIMA')
forecast_prophet.pd_series().plot(label='prophet')
plt.legend()

#### 4Theta

In [ ]:
freq = 'Yearly'
m = int(info_dataset.Frequency[freq[0]+'1'])
plt.figure()
ts_train = pickle.load(open("dataset/train_"+freq+".pkl", "rb"))
ts_test = pickle.load(open("dataset/test_"+freq+".pkl", "rb"))
_id = 1
train = ts_train[_id]
test = ts_test[_id]
fourtheta = FourTheta.select_best_model(train, thetas=[2], m=m)
theta = Theta(seasonality_period=m)
print(fourtheta)

fourtheta.fit(train)
theta.fit(train)

forecast_fourtheta = fourtheta.predict(len(test))
forecast_theta = theta.predict(len(test))

test.plot(label='test')
forecast_fourtheta.plot(label='4theta')
forecast_theta.plot(label='classic theta')
plt.title(freq + ' frequency')
print("Theta MASE: {:.3f}".format(np.mean(mase_m4(train, test, forecast_theta))))
print("4Theta MASE: {:.3f}".format(np.mean(mase_m4(train, test, forecast_fourtheta))))
plt.legend();

In [ ]:
freq = 'Quarterly'
m = int(info_dataset.Frequency[freq[0]+'1'])
plt.figure()
ts_train = pickle.load(open("dataset/train_"+freq+".pkl", "rb"))
ts_test = pickle.load(open("dataset/test_"+freq+".pkl", "rb"))
_id = 1
train = ts_train[_id]
test = ts_test[_id]
fourtheta = FourTheta.select_best_model(train, thetas=[2], m=m)
theta = Theta(seasonality_period=m)
print(fourtheta)

fourtheta.fit(train)
theta.fit(train)

forecast_fourtheta = fourtheta.predict(len(test))
forecast_theta = theta.predict(len(test))

test.plot(label='test')
forecast_fourtheta.plot(label='4theta')
forecast_theta.plot(label='classic theta')
plt.title(freq + ' frequency')
print("Theta MASE: {:.3f}".format(np.mean(mase_m4(train, test, forecast_theta))))
print("4Theta MASE: {:.3f}".format(np.mean(mase_m4(train, test, forecast_fourtheta))))
plt.legend();

In [ ]:
freq = 'Monthly'
m = int(info_dataset.Frequency[freq[0]+'1'])
plt.figure()
ts_train = pickle.load(open("dataset/train_"+freq+".pkl", "rb"))
ts_test = pickle.load(open("dataset/test_"+freq+".pkl", "rb"))
_id = 1
train = ts_train[_id]
test = ts_test[_id]
fourtheta = FourTheta.select_best_model(train, thetas=[2], m=m)
theta = Theta(seasonality_period=m)
print(fourtheta)

fourtheta.fit(train)
theta.fit(train)

forecast_fourtheta = fourtheta.predict(len(test))
forecast_theta = theta.predict(len(test))

test.plot(label='test')
forecast_fourtheta.plot(label='4theta')
forecast_theta.plot(label='classic theta')
plt.title(freq + ' frequency')
print("Theta MASE: {:.3f}".format(np.mean(mase_m4(train, test, forecast_theta))))
print("4Theta MASE: {:.3f}".format(np.mean(mase_m4(train, test, forecast_fourtheta))))
plt.legend();

In [ ]:
freq = 'Weekly'
m = int(info_dataset.Frequency[freq[0]+'1'])
plt.figure()
ts_train = pickle.load(open("dataset/train_"+freq+".pkl", "rb"))
ts_test = pickle.load(open("dataset/test_"+freq+".pkl", "rb"))
_id = 1
train = ts_train[_id]
test = ts_test[_id]
fourtheta = FourTheta.select_best_model(train, thetas=[2], m=m)
theta = Theta(seasonality_period=m)
print(fourtheta)

fourtheta.fit(train)
theta.fit(train)

forecast_fourtheta = fourtheta.predict(len(test))
forecast_theta = theta.predict(len(test))

test.plot(label='test')
forecast_fourtheta.plot(label='4theta')
forecast_theta.plot(label='classic theta')
plt.title(freq + ' frequency')
print("Theta MASE: {:.3f}".format(np.mean(mase_m4(train, test, forecast_theta))))
print("4Theta MASE: {:.3f}".format(np.mean(mase_m4(train, test, forecast_fourtheta))))
plt.legend();

In [ ]:
freq = 'Daily'
m = int(info_dataset.Frequency[freq[0]+'1'])
plt.figure()
ts_train = pickle.load(open("dataset/train_"+freq+".pkl", "rb"))
ts_test = pickle.load(open("dataset/test_"+freq+".pkl", "rb"))
_id = 1
train = ts_train[_id]
test = ts_test[_id]
fourtheta = FourTheta.select_best_model(train, thetas=[2], m=m)
theta = Theta(seasonality_period=m)
print(fourtheta)

fourtheta.fit(train)
theta.fit(train)

forecast_fourtheta = fourtheta.predict(len(test))
forecast_theta = theta.predict(len(test))

test.plot(label='test')
forecast_fourtheta.plot(label='4theta')
forecast_theta.plot(label='classic theta')
plt.title(freq + ' frequency')
print("Theta MASE: {:.3f}".format(np.mean(mase_m4(train, test, forecast_theta))))
print("4Theta MASE: {:.3f}".format(np.mean(mase_m4(train, test, forecast_fourtheta))))
plt.legend();

In [ ]:
freq = 'Hourly'
m = int(info_dataset.Frequency[freq[0]+'1'])
plt.figure()
ts_train = pickle.load(open("dataset/train_"+freq+".pkl", "rb"))
ts_test = pickle.load(open("dataset/test_"+freq+".pkl", "rb"))
_id = 0
train = ts_train[_id]
test = ts_test[_id]
fourtheta = FourTheta.select_best_model(train, thetas=[2], m=m)
theta = Theta(seasonality_period=m)
print(fourtheta)

fourtheta.fit(train)
theta.fit(train)

forecast_fourtheta = fourtheta.predict(len(test))
forecast_theta = theta.predict(len(test))

test.plot(label='test')
forecast_fourtheta.plot(label='4theta')
forecast_theta.plot(label='classic theta')
plt.title(freq + ' frequency')
print("Theta MASE: {:.3f}".format(np.mean(mase_m4(train, test, forecast_theta))))
print("4Theta MASE: {:.3f}".format(np.mean(mase_m4(train, test, forecast_fourtheta))))
plt.legend();

Now, let's try to find a better theta, and a better frequency

In [ ]:
freq = 'Weekly'
plt.figure()
ts_train = pickle.load(open("dataset/train_"+freq+".pkl", "rb"))
ts_test = pickle.load(open("dataset/test_"+freq+".pkl", "rb"))
_id = 2
train = ts_train[_id][-100:]
test = ts_test[_id]
fourtheta = FourTheta.select_best_model(train, thetas=np.linspace(-1, 10, 90), m=None)
theta = Theta(theta=(2 - fourtheta.theta))
print(fourtheta)

fourtheta.fit(train)
theta.fit(train)

forecast_fourtheta = fourtheta.predict(len(test))
forecast_theta = theta.predict(len(test))

test.plot(label='test')
forecast_fourtheta.plot(label='4theta')
forecast_theta.plot(label='classic theta')
plt.title(freq + ' frequency')
print("Theta MASE: {:.3f}".format(np.mean(mase_m4(train, test, forecast_theta))))
print("4Theta MASE: {:.3f}".format(np.mean(mase_m4(train, test, forecast_fourtheta))))
plt.legend();

#### run evaluation

In [ ]:
run_baseline = False
if run_baseline:
    %run -i "evaluate_baselines.py"

In [ ]:
run_thetas = False
if run_thetas:
    %run -i "evaluate_theta_methods.py"

In [ ]:
run_fft = False
if run_fft:
    %run -i "evaluate_fft.py"

In [ ]:
run_arima = False
if run_arima:
    %run -i "evaluate_arima.py"

In [ ]:
run_prophet = False
if run_prophet:
    %run -i "evaluate_prophet.py"

#### Ensembling

In [ ]:
from sklearn.linear_model import LassoCV

In [ ]:
from evaluate_ensembling import naive2_groe, groe_owa, DeseasonForecastingModel
deseason_model = DeseasonForecastingModel

In [ ]:
freq = 'Monthly'
ts_train = pickle.load(open("dataset/train_"+freq+".pkl", "rb"))
ts_test = pickle.load(open("dataset/test_"+freq+".pkl", "rb"))

mase_all = []
smape_all = []
m = int(info_dataset.Frequency[freq[0]+'1'])
for train, test in tqdm(zip(ts_train[:5], ts_test[:5])):
    # remove seasonality
    train_des=train
    seasonOut = 1
    season = ct(length=len(train), freq=train.freq_str, start_ts=train.start_time())
    if m > 1:
        if check_seasonality(train, m=int(m), max_lag=2*m):
            pass
            _, season = extract_trend_and_seasonality(train, m, model=SeasonalityMode.MULTIPLICATIVE)
            train_des = remove_seasonality(train, freq=m, model=SeasonalityMode.MULTIPLICATIVE)
            seasonOut = season[-m:].shift(m)
            seasonOut = seasonOut.append_values(seasonOut.values())[:len(test)]
    # model choice
    naiveSeason = NaiveSeasonal(K=m)
    naiveDrift = NaiveDrift()
    naive2 = NaiveSeasonal(K=1)
    ses = ExponentialSmoothing(trend=None, seasonal=None, seasonal_periods=m)
    holt = ExponentialSmoothing(seasonal=None, damped=False, trend='additive', seasonal_periods=m)
    damp = ExponentialSmoothing(seasonal=None, damped=True, trend='additive', seasonal_periods=m)
    # prophet = Prophet(yearly_seasonality=True, weekly_seasonality=False, daily_seasonality=False)
    
    fourtheta = FourTheta.select_best_model(train, thetas=[1, 2, 3], m=m)
    theta = Theta(theta=2, season_mode=SeasonalityMode.MULTIPLICATIVE, seasonality_period=m)
    models_simple = [naiveSeason, theta, fourtheta]
    models_des = [naive2, ses, holt, damp]

    # linear regression (with constraints)
    def train_pred(id_start=None, id_end=None):
        for m in models_simple:
            m.fit(train[id_start:id_end])
        for m in models_des:
            m.fit(train_des[id_start:id_end])
        models_simple_predictions = [m.predict(len(test))
                                     for m in models_simple]
        id_fin = id_end+len(test)
        if id_fin == 0:
            id_fin = None
        models_des_predictions = [m.predict(len(test)) * (seasonOut if id_end is None else season[id_end:id_fin])
                                  for m in models_des]

        model_predictions = models_simple_predictions + models_des_predictions
        
        return model_predictions
    
    val_predictions = train_pred(id_end=-len(test))

    regr_model = LinearRegressionModel(train_n_points=len(test), 
                                         model=LassoCV(positive=True, fit_intercept=False, max_iter=5000))
    target_val = train.slice_intersect(val_predictions[0])
    regr_model.fit(val_predictions, target_val)
    bktest_pred = val_predictions
    
    for mod in models_simple:
        mod.fit(train)
    for mod in models_des:
        mod.fit(train_des)
    
    models_simple_predictions = [mod.predict(len(test))
                                 for mod in models_simple]
    models_des_predictions = [mod.predict(len(test)) * seasonOut
                              for mod in models_des]

    model_predictions = models_simple_predictions + models_des_predictions
    
    regr_model.model.coef_ = regr_model.model.coef_/np.sum(regr_model.model.coef_)
    
    ensemble_pred = regr_model.predict(model_predictions)
    
    # Mean ensembling
    mean_pred = 0
    for pred in model_predictions:
        mean_pred = pred + mean_pred
    mean_pred = mean_pred/len(model_predictions)
    
    ## GROE OWA
    criterion = []
    criterion.append(groe_owa(train, naiveSeason, max(5, len(train)-len(test)), int(np.floor(len(test)/6)), 6, m))
    criterion.append(groe_owa(train, theta, max(5, len(train)-len(test)), int(np.floor(len(test)/6)), 6, m))
    criterion.append(groe_owa(train, fourtheta, max(5, len(train)-len(test)), int(np.floor(len(test)/6)), 6, m))
    criterion.append(groe_owa(train, deseason_model(NaiveSeasonal(K=1), m), max(5, len(train)-len(test)), int(np.floor(len(test)/6)), 6, m))
    criterion.append(groe_owa(train, deseason_model(ses, m), max(5, len(train)-len(test)), int(np.floor(len(test)/6)), 6, m))
    criterion.append(groe_owa(train, deseason_model(holt, m), max(5, len(train)-len(test)), int(np.floor(len(test)/6)), 6, m))
    criterion.append(groe_owa(train, deseason_model(damp, m), max(5, len(train)-len(test)), int(np.floor(len(test)/6)), 6, m))
    
    Score = 1/np.array(criterion)
    pesos = Score/Score.sum()
    
    groe_ensemble = 0
    for prediction, weight in zip(model_predictions, pesos):
        groe_ensemble = prediction * weight + groe_ensemble
    
    # BO3 ensembling
    score = np.argsort(Score)[::-1][:3]
    pesos2 = Score[score]/Score[score].sum()
    
    bo3_ensemble = 0
    bo3_mean = 0
    for i, model in enumerate(score):
            bo3_ensemble = model_predictions[model]*pesos2[i] + bo3_ensemble
            bo3_mean = model_predictions[model]/len(score) + bo3_mean
    
    mase_all.append(np.vstack([
                               mase_m4(train, test, models_des_predictions[0], m=m),
                               mase_m4(train, test, ensemble_pred, m=m),
                               mase_m4(train, test, mean_pred, m=m),
                               mase_m4(train, test, groe_ensemble, m=m),
                               mase_m4(train, test, bo3_ensemble, m=m),
                               mase_m4(train, test, bo3_mean, m=m),
                              ]))
    smape_all.append(np.vstack([
                                smape_m4(test, models_des_predictions[0]),
                                smape_m4(test, ensemble_pred),
                                smape_m4(test, mean_pred),
                                smape_m4(test, groe_ensemble),
                                smape_m4(test, bo3_ensemble),
                                smape_m4(test, bo3_mean),
                               ]))

In [ ]:
print("MASE; Naive2: {:.3f}, Linear Regression: {:.3f}, Mean ensembling: {:.3f}, GROE ensembling: {:.3f}, "
      "BO3 ensembling: {:.3f}, BO3 Mean: {:.3f}".format(*tuple(np.nanmean(np.stack(mase_all), axis=(0, 2)))))
print("sMAPE; Naive2: {:.3f}, Linear Regression: {:.3f}, Mean ensembling: {:.3f}, GROE ensembling: {:.3f}, "
      "BO3 ensembling: {:.3f}, BO3 Mean: {:.3f}".format(*tuple(np.nanmean(np.stack(smape_all), axis=(0, 2)))))

In [ ]:
print("OWA: ", owa_m4(freq,
                      np.nanmean(np.stack(mase_all), axis=(0, 2)),
                      np.nanmean(np.stack(smape_all), axis=(0, 2))))

#### Visualization

Linear Regression (Lasso)

In [ ]:
labels = ['naiveS', 'theta', '4theta', 'naive2','SES', 'holt', 'damped']
target_val.plot(label="target", lw=3)
pred = regr_model.predict(bktest_pred)
pred.plot(label="ensemble", lw=3)
for i, mod in enumerate(bktest_pred):
    mod.plot(label=labels[i])
plt.legend()
plt.title("Validation MASE = {:.3f}".format(np.mean(mase_m4(train[:-len(test)], train[-len(test):], pred, m=m))))

In [ ]:
regr_model.model.coef_

In [ ]:
labels = ['naiveS', 'theta', '4theta', 'naive2','SES', 'holt', 'damped']
test.plot(label="target", lw=3)
ensemble_pred.plot(label="ensemble", lw=3)
for i,mod in enumerate(model_predictions):
    mod.plot(label=labels[i])
plt.legend()
plt.title("Test MASE = {:.3f}".format(np.mean(mase_m4(train, test, ensemble_pred, m=m))))

GROE and other Ensembling

In [ ]:
test.plot(label="target", lw=3)
mean_pred.plot(label='mean ensemble', lw=3)
groe_ensemble.plot(label="groe ensemble", lw=3)
bo3_mean.plot(label="bo3 ensemble")
plt.legend()
plt.title("MASE mean = {:.3f}, GROE = {:.3f}".format(np.mean(mase_m4(train, test, mean_pred, m=m)), 
                                                     np.mean(mase_m4(train, test, groe_ensemble, m=m))))

In [ ]:
# GROE weights
pesos